In [11]:
import pandas as pd
import numpy as np
import json
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import dateutil.parser as parser
from datetime import datetime, date, timedelta
import torch
import skorch
import scipy
import torch.nn as nn
import torch
import torch.nn.functional as F
import sys
from skorch.helper import DataFrameTransformer
import time
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, mean_squared_error, r2_score
from sklearn import metrics
from sklearn.preprocessing import FunctionTransformer
from skorch.callbacks import EarlyStopping
from sklearn.pipeline import Pipeline
from skorch import NeuralNetRegressor


# Please ignore the below cells, they were for additional preprocessing and do not need to be run again

In [54]:
# Opening JSON file
f = open('twitter_aapl_1.json',)
  
# returns JSON object as 
# a dictionary
aapl = json.load(f)

f.close()

In [55]:
flataapl  = [val for sublist in aapl for val in sublist]

In [56]:
print(flataapl[0])

{'created_at': '2021-05-11T10:18:35.000Z', 'id': '1392061565185236994', 'text': 'RT @Nicochan33: Apple Execs Chose to Keep a Hack of 128 Million iPhones Quiet https://t.co/waR9tgHKCA #tech #feedly #apple #iphone #cyberse…'}


In [57]:
df = pd.DataFrame(flataapl)
df

,created_at,id,text,withheld
0,2021-05-11T10:18:35.000Z,1392061565185236994,RT @Nicochan33: Apple Execs Chose to Keep a Ha...,NaN
1,2021-05-11T10:18:30.000Z,1392061546751217671,"RT @RoseLoveStyle: House of the Dragon, the pr...",NaN
2,2021-05-11T10:18:25.000Z,1392061526614360066,RT @gtorges: Ich habe jetzt einiges an Materia...,NaN
3,2021-05-11T10:17:54.000Z,1392061393415909376,RT @iTech911: Future versions of #Apple's CarK...,NaN
4,2021-05-11T10:17:48.000Z,1392061371886485506,発売日に予約してようやく届いた🤗\n#Apple #airtag #ケースは純正じゃない #...,NaN
...,...,...,...,...
517126,2021-02-18T07:25:14.000Z,1362302135333969929,RT @mobitrade_: Original Apple Airpods Pro New...,NaN
517127,2021-02-18T07:25:08.000Z,1362302111027929089,#ios #Apple #swiftui \nSwiftUI 2.0 Complex UI ...,NaN
517128,2021-02-18T07:25:06.000Z,1362302102404542467,Sommige appels worden te snel slecht.\n\nhttps...,NaN
517129,2021-02-18T07:24:56.000Z,1362302061589708805,#apple #kia #StockMarkets https://t.co/ccRBBAKDtd,NaN


In [58]:
y = df['withheld'].dropna()
y

152369    {'copyright': False, 'country_codes': ['IN']}
257375    {'copyright': False, 'country_codes': ['IN']}
269997    {'copyright': False, 'country_codes': ['IN']}
315545    {'copyright': False, 'country_codes': ['RU']}
315578    {'copyright': False, 'country_codes': ['RU']}
343775    {'copyright': False, 'country_codes': ['IN']}
473481    {'copyright': False, 'country_codes': ['RU']}
473719    {'copyright': False, 'country_codes': ['RU']}
489545    {'copyright': False, 'country_codes': ['IN']}
509238    {'copyright': False, 'country_codes': ['IN']}
Name: withheld, dtype: object

In [59]:
df1 = df.drop('withheld',axis=1)
df1['created_at'] = df1['created_at'].map(lambda x: str(x)[:-5])
df1

,created_at,id,text
0,2021-05-11T10:18:35,1392061565185236994,RT @Nicochan33: Apple Execs Chose to Keep a Ha...
1,2021-05-11T10:18:30,1392061546751217671,"RT @RoseLoveStyle: House of the Dragon, the pr..."
2,2021-05-11T10:18:25,1392061526614360066,RT @gtorges: Ich habe jetzt einiges an Materia...
3,2021-05-11T10:17:54,1392061393415909376,RT @iTech911: Future versions of #Apple's CarK...
4,2021-05-11T10:17:48,1392061371886485506,発売日に予約してようやく届いた🤗\n#Apple #airtag #ケースは純正じゃない #...
...,...,...,...
517126,2021-02-18T07:25:14,1362302135333969929,RT @mobitrade_: Original Apple Airpods Pro New...
517127,2021-02-18T07:25:08,1362302111027929089,#ios #Apple #swiftui \nSwiftUI 2.0 Complex UI ...
517128,2021-02-18T07:25:06,1362302102404542467,Sommige appels worden te snel slecht.\n\nhttps...
517129,2021-02-18T07:24:56,1362302061589708805,#apple #kia #StockMarkets https://t.co/ccRBBAKDtd


In [60]:
df1.to_csv("twitter_aapl_1_clean.csv", index = False)

In [2]:
df1 = pd.read_csv("twitter_aapl_1_clean.csv")
# Only including this step for now, basically removes the time component, leaving only date
df1['created_at'] = df1['created_at'].map(lambda x: str(x)[:-9])
df1 = df1.rename(columns={"created_at" : "Date"})
df1

,Date,id,text
0,2021-05-11,1392061565185236994,RT @Nicochan33: Apple Execs Chose to Keep a Ha...
1,2021-05-11,1392061546751217671,"RT @RoseLoveStyle: House of the Dragon, the pr..."
2,2021-05-11,1392061526614360066,RT @gtorges: Ich habe jetzt einiges an Materia...
3,2021-05-11,1392061393415909376,RT @iTech911: Future versions of #Apple's CarK...
4,2021-05-11,1392061371886485506,発売日に予約してようやく届いた🤗\n#Apple #airtag #ケースは純正じゃない #...
...,...,...,...
517126,2021-02-18,1362302135333969929,RT @mobitrade_: Original Apple Airpods Pro New...
517127,2021-02-18,1362302111027929089,#ios #Apple #swiftui \nSwiftUI 2.0 Complex UI ...
517128,2021-02-18,1362302102404542467,Sommige appels worden te snel slecht.\n\nhttps...
517129,2021-02-18,1362302061589708805,#apple #kia #StockMarkets https://t.co/ccRBBAKDtd


In [3]:
# adapted from https://www.30secondsofcode.org/python/s/is-weekend
def if_weekend(d = datetime.today()):
    if d.weekday() == 5:
        da = d + timedelta(days=2)
        return da
    elif d.weekday() == 6:
        da = d + timedelta(days=1)
        return da
    elif d == date(2021,4,2):
        return date(2021,4,5)
    else:
        return d
print(if_weekend(date(2021,6,14)))
print(type(date(2021,6,14)))

2021-06-14
<class 'datetime.date'>


In [4]:
df1['Date']= pd.to_datetime(df1['Date'])
df1['usedate'] = df1['Date'].map(lambda x: if_weekend(x))
df1

,Date,id,text,usedate
0,2021-05-11,1392061565185236994,RT @Nicochan33: Apple Execs Chose to Keep a Ha...,2021-05-11
1,2021-05-11,1392061546751217671,"RT @RoseLoveStyle: House of the Dragon, the pr...",2021-05-11
2,2021-05-11,1392061526614360066,RT @gtorges: Ich habe jetzt einiges an Materia...,2021-05-11
3,2021-05-11,1392061393415909376,RT @iTech911: Future versions of #Apple's CarK...,2021-05-11
4,2021-05-11,1392061371886485506,発売日に予約してようやく届いた🤗\n#Apple #airtag #ケースは純正じゃない #...,2021-05-11
...,...,...,...,...
517126,2021-02-18,1362302135333969929,RT @mobitrade_: Original Apple Airpods Pro New...,2021-02-18
517127,2021-02-18,1362302111027929089,#ios #Apple #swiftui \nSwiftUI 2.0 Complex UI ...,2021-02-18
517128,2021-02-18,1362302102404542467,Sommige appels worden te snel slecht.\n\nhttps...,2021-02-18
517129,2021-02-18,1362302061589708805,#apple #kia #StockMarkets https://t.co/ccRBBAKDtd,2021-02-18


In [5]:
price = pd.read_csv("aapl2.csv")
price['Date'] = price['Date'].map(lambda x: parser.parse(x).isoformat())
# Only including this step for now, basically removes the time component, leaving only date
price['Date'] = price['Date'].map(lambda x: str(x)[:-9])
price['Date']= pd.to_datetime(price['Date'])
# Change me if looking to change examination scope
price = price.rename(columns={"Date" : "usedate"})
price

,usedate,Close/Last,Volume,Open,High,Low,Change
0,2021-05-11,125.9100,126142800,123.5000,126.2700,122.7700,2.4100
1,2021-05-10,126.8500,88071230,129.4100,129.5400,126.8100,-2.5600
2,2021-05-07,130.2100,78973270,130.8500,131.2582,129.4750,-0.6400
3,2021-05-06,129.7400,78128330,127.8900,129.7500,127.1300,1.8500
4,2021-05-05,128.1000,84000900,129.2000,130.4500,127.9700,-1.1000
...,...,...,...,...,...,...,...
247,2020-05-18,78.7400,135372520,78.2925,79.1250,77.5810,0.4475
248,2020-05-15,76.9275,166348360,75.0875,76.9750,75.0525,1.8400
249,2020-05-14,77.3850,158929080,76.1275,77.4475,75.3825,1.2575
250,2020-05-13,76.9125,200622560,78.0375,78.9875,75.8025,-1.1250


In [6]:
merged = pd.merge(df1, price, how='left', on='usedate')
merged

,Date,id,text,usedate,Close/Last,Volume,Open,High,Low,Change
0,2021-05-11,1392061565185236994,RT @Nicochan33: Apple Execs Chose to Keep a Ha...,2021-05-11,125.91,126142800,123.5,126.270,122.77,2.41
1,2021-05-11,1392061546751217671,"RT @RoseLoveStyle: House of the Dragon, the pr...",2021-05-11,125.91,126142800,123.5,126.270,122.77,2.41
2,2021-05-11,1392061526614360066,RT @gtorges: Ich habe jetzt einiges an Materia...,2021-05-11,125.91,126142800,123.5,126.270,122.77,2.41
3,2021-05-11,1392061393415909376,RT @iTech911: Future versions of #Apple's CarK...,2021-05-11,125.91,126142800,123.5,126.270,122.77,2.41
4,2021-05-11,1392061371886485506,発売日に予約してようやく届いた🤗\n#Apple #airtag #ケースは純正じゃない #...,2021-05-11,125.91,126142800,123.5,126.270,122.77,2.41
...,...,...,...,...,...,...,...,...,...,...
517126,2021-02-18,1362302135333969929,RT @mobitrade_: Original Apple Airpods Pro New...,2021-02-18,129.71,96856750,129.2,129.995,127.41,0.51
517127,2021-02-18,1362302111027929089,#ios #Apple #swiftui \nSwiftUI 2.0 Complex UI ...,2021-02-18,129.71,96856750,129.2,129.995,127.41,0.51
517128,2021-02-18,1362302102404542467,Sommige appels worden te snel slecht.\n\nhttps...,2021-02-18,129.71,96856750,129.2,129.995,127.41,0.51
517129,2021-02-18,1362302061589708805,#apple #kia #StockMarkets https://t.co/ccRBBAKDtd,2021-02-18,129.71,96856750,129.2,129.995,127.41,0.51


In [7]:
merged.isna().sum()

Date          0
id            0
text          0
usedate       0
Close/Last    0
Volume        0
Open          0
High          0
Low           0
Change        0
dtype: int64

In [24]:
# merged.to_csv("merged.csv", index = False)

# Start Here

In [13]:
merged = pd.read_csv('mergedfull.csv')
from sklearn.model_selection import train_test_split  
mergedbig, mergedsmall = train_test_split(merged, test_size=0.15, random_state=0)
# mergedsmall.to_csv("mergedsmall.csv", index = False)
mergedsmall = pd.read_csv('mergedsmall.csv')
merged = mergedsmall
# Obtaining the tweet contents into a list
all_tweets = merged["text"]
print(all_tweets)
all_tweets = all_tweets.to_list()
print(all_tweets[0])
print(all_tweets[1])
print(all_tweets[2])

                                                      text  referenced_tweets  \
0        .\nDomain Name for sale\n\nhttps://t.co/f056Wp...       1.391353e+18   
1        .\nDomain NameFor Sale\n\nhttps://t.co/a2N3rgJ...       1.391354e+18   
2        .\nDomains For sale\n\nhttps://t.co/0YeRs3JJLb...       1.391353e+18   
3        .\nDomain NameFor Sale\n\nhttps://t.co/a2N3rgJ...       1.391354e+18   
4        Savanna Drinkers 🤦\n\n#R500 #RHOA #AceMagashul...       1.392030e+18   
...                                                    ...                ...   
3387871  Download JOBIN THE PENGUIN! Awesome, retro-ins...                NaN   
3387872  2019 13" 1.6GHz/128GB #Apple #MacBook Airs, re...                NaN   
3387873  How to use Apple Maps mobility trends data htt...                NaN   
3387874  AirPods Studio Could Come With Head And Neck D...                NaN   
3387875  Check it OUT Shades of Gray by Andy Holloman R...                NaN   

                          i

In [3]:
print(len(all_tweets))

508182


In [4]:
import nltk
# nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
lemmatizer = WordNetLemmatizer()
def lemming(tweet):
    a = word_tokenize(tweet)
    answer = list(map(lambda x: lemmatizer.lemmatize(x), a))
    return answer
from nltk.tokenize.treebank import TreebankWordDetokenizer

In [5]:
# https://python.gotrained.com/scraping-tweets-sentiment-analysis/
# Basic cleaning of text before TF-IDF; this process will be improved later
# for tweet in all_tweets:
#     # Remove all the special characters
#     processed_tweet = re.sub(r'\W', ' ', tweet)
 
#     # remove all single characters
#     processed_tweet = re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_tweet)
 
#     # Remove single characters from the start
#     processed_tweet = re.sub(r'\^[a-zA-Z]\s+', ' ', processed_tweet) 
 
#     # Substituting multiple spaces with single space
#     processed_tweet= re.sub(r'\s+', ' ', processed_tweet, flags=re.I)
 
#     # Removing prefixed 'b'
#     processed_tweet = re.sub(r'^b\s+', '', processed_tweet)
    
#     #removing common pronouns and prepositions
#     processed_tweet = re.sub(r'of|to|https|keep|128', '', processed_tweet)
#     processed_tweet = lemmatizer.lemmatize(processed_tweet)
# #     lambda x: lemmatizer.lemmatize(x in processed_tweet)
#     # Converting to Lowercase
#     processed_tweet = processed_tweet.lower()
    
#     tweet = processed_tweet
    
    
processed_tweets = []
X = all_tweets
for tweet in range(0, len(X)):  
    

    processed_tweet = re.sub(r"http\S+", "", str(X[tweet]))

    # Remove all the special characters
    processed_tweet = re.sub(r'\W', ' ', processed_tweet)
    
#     processed_tweet = re.sub(r'http\S+', '', processed_tweet)
    
    
#     processed_tweet = re.sub(r'co\S+', '', processed_tweet) 
    # remove all single characters
    processed_tweet = re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_tweet)
 
    # Remove single characters from the start
    processed_tweet = re.sub(r'\^[a-zA-Z]\s+', ' ', processed_tweet) 
 
    # Substituting multiple spaces with single space
    processed_tweet= re.sub(r'\s+', ' ', processed_tweet, flags=re.I)
 
    # Removing prefixed 'b'
    processed_tweet = re.sub(r'^b\s+', '', processed_tweet)
    

    
#     processed_tweet = re.sub(r'of|to|https|keep', '', processed_tweet)
    # Converting to Lowercase
    processed_tweet = processed_tweet.lower()
    
    processed_tweet = lemming(processed_tweet)
    processed_tweet = TreebankWordDetokenizer().detokenize(processed_tweet)    
    
    # remove all single characters
    processed_tweet = re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_tweet)
    processed_tweets.append(processed_tweet)

In [6]:
print(processed_tweets[0])
print(processed_tweets[1])
print(processed_tweets[2])

2020 is almost ended brand new sealed factory unlocked iphone12 mini to one lucky follower keep following amp retweeting itech911 ibuy isell iswap ifix apple 0262666226
if you feel like apple ha no love for you by not delivering the heart month challenge badge just restart your watch and wait for few minute you ll get it apple applewatch challenge
new rip is live 2w1b 051 iphone capitalism amazon tax gripe mondaythoughts mondaymorning mondaymotivation hamont btc bitcoin iphone12 apple aapl eeftober spotify ps5 investing itunes sony weed cgc etf amzn spot nlpoli


In [7]:

def identity_tokenizer(text):
    return text
tfidfv = TfidfVectorizer(tokenizer=identity_tokenizer, input="array", lowercase=False, norm="l2", max_features=None, sublinear_tf=True, stop_words="english",token_pattern=r'[^\s]+')
df2 = tfidfv.fit_transform(processed_tweets)
print(df2)
# df2array = df2.toarray()
# print(df2array)

D:\Documents\anaconda3\envs\DataSciEnv\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'd', 'e', 'f', 'g', 'h', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


  (0, 7)	0.4424314246336387
  (0, 33)	0.1249567979802098
  (0, 10)	0.16582392918790628
  (0, 17)	0.11262468721438612
  (0, 2)	0.2080409113503352
  (0, 18)	0.10805227545782864
  (0, 25)	0.14608843790319945
  (0, 20)	0.18189356622112837
  (0, 30)	0.14506418279955546
  (0, 34)	0.16969490368608672
  (0, 13)	0.1531901222681532
  (0, 16)	0.17256851227177736
  (0, 32)	0.20371176864288223
  (0, 27)	0.1355941884983446
  (0, 12)	0.1266146098596461
  (0, 14)	0.15771445039678494
  (0, 23)	0.18061428764535165
  (0, 15)	0.20058649668753167
  (0, 29)	0.16542284403544966
  (0, 24)	0.18446483736773406
  (0, 22)	0.13564614142985149
  (0, 21)	0.17761529804847354
  (0, 28)	0.15788935531951526
  (0, 0)	0.22089715206910612
  (0, 1)	0.2514566660491283
  :	:
  (508180, 1)	0.041268949921978985
  (508180, 3)	0.06685152242931619
  (508181, 5)	0.20455669475441884
  (508181, 31)	0.11378430497461173
  (508181, 17)	0.21664203981153166
  (508181, 18)	0.2682199050847498
  (508181, 25)	0.18636512734943322
  (508181, 20

In [21]:
print(type(df2))
print(type(df2[-1]))
print(type(df2[-1][-1]))
print(df2[-1][-1])

<class 'scipy.sparse.csr.csr_matrix'>
<class 'scipy.sparse.csr.csr_matrix'>
<class 'scipy.sparse.csr.csr_matrix'>
  (0, 556)	0.40357727068148264
  (0, 88)	0.34308710016633726
  (0, 97)	0.417333363196858
  (0, 443)	0.3951839877481259
  (0, 674)	0.3663719517401211
  (0, 1770)	0.3199229308458355
  (0, 1783)	0.31050472625527564
  (0, 943)	0.15704072326243243
  (0, 1716)	0.15129041849415864
  (0, 1471)	0.09239185493528781


In [11]:
print(df2.size)

6813202


In [12]:
print(df2[0])

print(df2array[0])
print(type(df2[0][0]))

  (0, 943)	0.1619164850216469
  (0, 1667)	0.24642359750991072
  (0, 412)	0.07984327940748012
  (0, 863)	0.07926270652557654
  (0, 957)	0.3644418579168204
  (0, 1148)	0.37252279088881424
  (0, 24)	0.44364094790152897
  (0, 1243)	0.19111331976332038
  (0, 802)	0.4405447542842643
  (0, 1005)	0.38259748006594846
  (0, 1716)	0.15598764620493955
  (0, 189)	0.14697293203863546
  (0, 1471)	0.09526041452797107
[0. 0. 0. ... 0. 0. 0.]
<class 'scipy.sparse.csr.csr_matrix'>


In [ ]:
print(type(df2array))
np.set_printoptions(threshold=sys.maxsize)
print(df2array[0])
np.set_printoptions(threshold = False)

In [20]:
print(processed_tweets[0:10])

['rt nicochan33 apple exec chose to keep hack of 128 million iphones quiet tech feedly apple iphone cyberse', 'rt roselovestyle house of the dragon the prequel to game of throne wa announced by casey bloys president of hbo gamesofthrones', 'rt gtorges ich habe jetzt einiges an material zu fuellmich amp co beim applesupport eingereicht hoffentlich verschwindet der podcast', 'rt itech911 future version of apple carkey could detect when it being used near wireless charger and alter how it work to avoid', '発売日に予約してようやく届いた apple airtag ケースは純正じゃない 忘れん坊 実は追加でもう一つ買った', 'rt nicochan33 apple exec chose to keep hack of 128 million iphones quiet tech feedly apple iphone cyberse', 'rt foxconcours_ apple macbook tente de gagner un macbookpro13 en participant ce concours pour tenter votre chance rt fo', 'rt taisy0 apple apple tv でsiriを利用可能な国を拡大 気になる 記になる apple', 'rt neodiycom how to fix apple macbookpro 2011 running windows10 10 64 bit audio sound driver problem macbook', 'rt indievideogames dino mak

In [22]:
print(len(processed_tweets))
print(type(processed_tweets[0]))

517131
<class 'str'>


In [15]:
print(type(df2))

<class 'scipy.sparse.csr.csr_matrix'>


In [6]:
# https://programmerbackpack.com/tf-idf-explained-and-python-implementation/
dfnew = pd.DataFrame(df2[0].T.todense(), index=tfidfv.get_feature_names(), columns=["TF-IDF"])
dfnew = dfnew.sort_values('TF-IDF', ascending=False)
print (dfnew.head(30))

NameError: name 'df2' is not defined

In [39]:
# Trying an alternative data implementation, I haven't used this in the later stages yet
df3 = pd.DataFrame.sparse.from_spmatrix(df2)
print(type(df3))
df3

<class 'pandas.core.frame.DataFrame'>


,0,1,2,3,4,5,6,7,8,9,...,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517126,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
517127,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
517128,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
517129,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
import pickle
#Selecting the data and splitting into train and test
y = merged['Change']
print(type(y))
# X = df3
# X = df2
# X = df2array
X = processed_tweets
from sklearn.model_selection import train_test_split  
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
y_train = y_train.values.reshape(-1,1)
y_test = y_test.values.reshape(-1,1)
y_train = y_train.astype(np.float32)
y_test = y_test.astype(np.float32)
# train_data = train_data.sample(frac=x)
# train_holdout_data.to_csv('train_holdout_data.csv', index=False)
# x_holdout.to_csv('x_holdout.csv', index=False)
# y_holdout.to_csv('y_holdout.csv', index=False)
# train_data.to_csv('train_data.csv', index=False)

# with open("x_train.txt", "wb") as fp:   #Pickling
#     pickle.dump(x_train, fp)
# with open("y_train.txt", "wb") as fp:   #Pickling
#     pickle.dump(y_train, fp)
# with open("x_test.txt", "wb") as fp:   #Pickling
#     pickle.dump(x_test, fp)
# with open("y_test.txt", "wb") as fp:   #Pickling
#     pickle.dump(y_test, fp)

with open("x_train.txt", "rb") as fp:   # Unpickling
    x_train = pickle.load(fp)
with open("y_train.txt", "rb") as fp:   # Unpickling
    y_train = pickle.load(fp)
with open("x_test.txt", "rb") as fp:   # Unpickling
    x_test = pickle.load(fp)
with open("y_test.txt", "rb") as fp:   # Unpickling
    y_test = pickle.load(fp)

# x_train.to_csv('x_train.csv', index=False)
# y_train.to_csv('y_train.csv', index=False)
# x_test.to_csv('x_test.csv', index=False)
# y_test.to_csv('y_test.csv', index=False)

# # # train_holdout_data = pd.read_csv('train_holdout_data.csv')
# # # x_holdout = pd.read_csv('x_holdout.csv')
# # # y_holdout = pd.read_csv('y_holdout.csv')
# x_train = pd.read_csv('x_train.csv')
# y_train = pd.read_csv('y_train.csv')
# x_test = pd.read_csv('x_test.csv')
# y_test = pd.read_csv('y_test.csv')


<class 'pandas.core.series.Series'>


In [16]:
# y_train = pd.Series(y_train)
# y_test = pd.Series(y_test)
print(len(x_train))
# print(type(y_train.values))
# print(np.isnan(X).sum())
# print(y.isna().sum())

406545


In [17]:
# x_train = torch.from_numpy(x_train)
# x_test = torch.from_numpy(x_test)
# y_train = torch.from_numpy(y_train.values)
# y_test = torch.from_numpy(y_test.values)
print(len(x_train))
# print(x_train)
# print(x_test)
# print(y_train)
# print(y_test)

406545


In [18]:
print(type(x_train))
print(type(x_test))
print(type(y_train))
print(type(y_test))
print(x_train.dtype)
print(x_test.dtype)
print(y_train.dtype)
print(y_test.dtype)
# print(x_test[100000][0])
# if 'X' in x_train:
#     print("hi")

<class 'list'>
<class 'list'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


AttributeError: 'list' object has no attribute 'dtype'

In [20]:
# y_train = y_train.astype(np.float32)
# y_test = y_test.astype(np.float32)
y_train = y_train.values.reshape(-1,1)
y_test = y_test.values.reshape(-1,1)
y_train = y_train.astype(np.float32)
y_test = y_test.astype(np.float32)
# y_train = y_train.squeeze()
# y_test = y_test.squeeze()

In [13]:
print(y_train.ndim)

2


In [43]:
# x_train1 = x_train.astype(np.float32)
# y_train1 = y_train.astype(np.float32)
print(x_train.dtype)
print(x_test.dtype)
print(y_train.dtype)
print(y_test.dtype)
print(x_train1.dtype)
print(y_train1.dtype)

AttributeError: 'list' object has no attribute 'dtypes'

In [45]:
# for x in y_train[0]:
#     if type(x) != "class 'numpy.float32'":
#         print("double")
print(type(x_train[0]))
print(type(y_train))
print(type(y_train[0]))
print(type(y_train[0][0]))

<class 'str'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.float32'>


AttributeError: 'numpy.ndarray' object has no attribute 'float'

In [20]:
y_trainarray = y_train.values.tolist()
y_testarray = y_test.values.tolist()
# y_trainarray = y_train.to_numpy()
# y_testarray = y_test.to_numpy()

In [19]:
def identity_tokenizer(text):
    return text

tfidfv = TfidfVectorizer(tokenizer=identity_tokenizer, input="array", lowercase=False, norm="l2", max_features=None, sublinear_tf=True, stop_words="english",token_pattern=r'[^\s]+')
x_train1 = tfidfv.fit_transform(x_train)
x_test1 = tfidfv.fit_transform(x_test)
x_train2 = x_train1.astype(dtype = np.float32)
x_test2 = x_test1.astype(dtype = np.float32)

D:\Documents\anaconda3\envs\DataSciEnv\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'd', 'e', 'f', 'g', 'h', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


In [14]:
print(x_train2.shape[1])
print(y_train.shape)
print(y_train)

5205
(413704, 1)
[[ 1.33]
 [-2.01]
 [ 1.14]
 ...
 [-3.29]
 [-2.73]
 [-1.33]]


In [33]:
print(len(x_train))
# tokenise me pls
a = [word_tokenize(x) for x in x_train]

In [39]:
def flatten(t):
    return [item for sublist in t for item in sublist]
print(len(set(flatten(a))))

373108


In [20]:
# import torch.nn.functional as F
from nltk.tokenize.treebank import TreebankWordDetokenizer
def identity_tokenizer(text):
    return text

in_dimension = 413704
hid_dimension = 3
out_dimension = 1


class PoleNN(nn.Module):
    def __init__(self):
        super(PoleNN, self).__init__()
        self.fc1 = nn.Linear(in_dimension,hid_dimension)
        self.fc2 = nn.Linear(hid_dimension,out_dimension)
        self.sigmoid = torch.nn.Sigmoid()
        
    def forward(self, X):
        hidden = self.fc1(X)
        hidden = self.sigmoid(hidden)
        output = self.fc2(hidden)
        return output

from skorch import NeuralNetRegressor
from sklearn.pipeline import Pipeline
x_trainshape = 5842
class RegressorModule(nn.Module):
    def __init__(
            self,
            num_units=10,
            nonlin=F.relu,
    ):
        super(RegressorModule, self).__init__()
        self.num_units = num_units
        self.nonlin = nonlin

        self.dense0 = nn.Linear(x_trainshape, num_units)
        self.nonlin = nonlin
        self.dense1 = nn.Linear(num_units, 10)
        self.output = nn.Linear(10, 1)

    def forward(self, X, **kwargs):
        X = self.nonlin(self.dense0(X))
        X = F.relu(self.dense1(X))
        X = self.output(X)
        return X

pole_model = RegressorModule()
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(pole_model.parameters(), lr = 0.1)

net = NeuralNetRegressor(module=pole_model, max_epochs=30, lr=0.1, callbacks =[('earlystopping',EarlyStopping())])
def typechange(x):
    return x.astype(dtype = np.float32)
typetransform = FunctionTransformer(typechange)
def inputneuron(x):
    x_trainshape = x.shape[1]
#     return x_trainshape
inputneuronnumber = FunctionTransformer(inputneuron)
# pipe = Pipeline([('transform', DataFrameTransformer()),('net', net)])
# pipe = Pipeline([('net', net)])
pipe = Pipeline([("tfidf_vector_com", TfidfVectorizer(tokenizer=identity_tokenizer, input="array", lowercase=False, norm="l2", max_features=None, sublinear_tf=True, stop_words="english",token_pattern=r'[^\s]+')), ("typetransform", typetransform), ("net", net)])


# net = skorch.NeuralNetClassifier(module=PoleNN, max_epochs=20, lr=0.1, criterion=torch.nn.NLLLoss)
pipe.fit(X=x_train, y=y_train)
y_pred = pipe.predict(x_test)

  epoch    train_loss    valid_loss      dur
-------  ------------  ------------  -------
      1        4.1850        4.0957  70.8741
      2        4.0236        4.0082  70.5556
      3        3.9830        3.9926  73.4007
      4        3.9650        3.9773  68.1609
      5        3.9520        3.9899  70.5284
      6        3.9411        3.9743  67.9473
      7        3.9332        3.9756  67.6246
      8        3.9267        3.9669  65.6628
      9        3.9200        3.9674  64.3621
     10        3.9137        3.9612  65.7217
     11        3.9090        3.9527  70.4800
     12        3.9033        3.9524  73.7520
     13        3.8986        3.9497  73.8936
     14        3.8937        3.9524  73.8280
     15        3.8897        3.9455  74.1685
     16        3.8851        3.9444  74.1716
     17        3.8813        3.9429  74.1616
     18        3.8762        3.9444  69.6191
     19        3.8730        3.9428  68.3819
     20        3.8691        3.9409  67.4868
     21   

In [22]:
rmse = mean_squared_error(y_test, y_pred, squared = False)
rmse

1.971518

In [12]:
optimum_params = {'net__lr': 0.0015539382932245303, 'net__max_epochs': 20, 'net__optimizer__momentum': 0.9375124227337543}
num_unitslist = [5,10,20,40,60,80,100]
rmselist = []
x_trainshape = 5842
for num in num_unitslist:
    class RegressorModule(nn.Module):
        def __init__(
                self,
                num_units=num,
                nonlin=F.relu,
        ):
            super(RegressorModule, self).__init__()
            self.num_units = num_units
            self.nonlin = nonlin

            self.dense0 = nn.Linear(x_trainshape, num_units)
            self.nonlin = nonlin
            self.dense1 = nn.Linear(num_units, num)
            self.output = nn.Linear(num, 1)

        def forward(self, X, **kwargs):
            X = self.nonlin(self.dense0(X))
            X = F.relu(self.dense1(X))
            X = self.output(X)
            return X

    pole_model = RegressorModule()

    net = NeuralNetRegressor(module=pole_model, max_epochs=optimum_params['net__max_epochs'], lr=optimum_params['net__lr'], optimizer__momentum=optimum_params['net__optimizer__momentum'], callbacks =[('earlystopping',EarlyStopping())])
    def typechange(x):
        return x.astype(dtype = np.float32)
    typetransform = FunctionTransformer(typechange)
    def inputneuron(x):
        x_trainshape = x.shape[1]
    #     return x_trainshape
    inputneuronnumber = FunctionTransformer(inputneuron)
    # pipe = Pipeline([('transform', DataFrameTransformer()),('net', net)])
    # pipe = Pipeline([('net', net)])
    pipe = Pipeline([("tfidf_vector_com", TfidfVectorizer(tokenizer=identity_tokenizer, input="array", lowercase=False, norm="l2", max_features=None, sublinear_tf=True, stop_words="english",token_pattern=r'[^\s]+')), ("typetransform", typetransform), ("net", net)])


    # net = skorch.NeuralNetClassifier(module=PoleNN, max_epochs=20, lr=0.1, criterion=torch.nn.NLLLoss)
    pipe.fit(X=x_train, y=y_train)
    y_pred = pipe.predict(x_test)
    rmse = mean_squared_error(y_test, y_pred, squared = False)
    print(num,'RMSE:', rmse)
    rmselist.append(rmse)

min_index = rmselist.index((min(rmselist)))
print("Optimum parameters:", num_unitslist[min_index],":",min(rmselist))

  epoch    train_loss    valid_loss      dur
-------  ------------  ------------  -------
      1        4.2394        4.2297  72.4814
      2        4.1567        4.1073  70.6289
      3        4.0670        4.0913  72.2662
      4        4.0426        4.0363  70.4901
      5        4.0162        4.0882  70.8557
      6        4.0003        4.0989  70.7887
      7        3.9910        4.0867  71.7170
      8        3.9837        4.0710  70.4721
Stopping since valid_loss has not improved in the last 5 epochs.
5 RMSE: 2.0023932


D:\Documents\anaconda3\envs\DataSciEnv\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'd', 'e', 'f', 'g', 'h', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


  epoch    train_loss    valid_loss      dur
-------  ------------  ------------  -------
      1        4.2401        4.2351  67.6599
      2        4.1687        4.0918  69.9882
      3        4.0694        4.0911  70.2413
      4        4.0430        4.0353  67.6599
      5        4.0176        4.0835  67.9217
      6        4.0010        4.0947  68.4621
      7        3.9883        4.0795  67.3682
      8        3.9796        4.0551  68.7278
Stopping since valid_loss has not improved in the last 5 epochs.
10 RMSE: 1.9989355


D:\Documents\anaconda3\envs\DataSciEnv\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'd', 'e', 'f', 'g', 'h', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


  epoch    train_loss    valid_loss      dur
-------  ------------  ------------  -------
      1        4.2416        4.2414  71.0854
      2        4.1907        4.1663  71.9156
      3        4.0650        4.0707  71.7418
      4        4.0051        4.0529  73.6199
      5        3.9785        4.0387  74.5445
      6        3.9599        4.0109  74.4860
      7        3.9452        4.0011  72.7338
      8        3.9340        3.9897  74.2163
      9        3.9245        3.9910  73.9905
     10        3.9156        3.9788  77.6048
     11        3.9071        3.9715  80.6734
     12        3.8989        3.9742  79.0142
     13        3.8918        3.9690  73.4953
     14        3.8852        3.9654  76.5282
     15        3.8782        3.9570  76.6049
     16        3.8717        3.9564  76.4651
     17        3.8648        3.9569  76.1926
     18        3.8588        3.9553  75.4409
     19        3.8527        3.9587  72.4091
     20        3.8469        3.9591  73.7127
20 RMSE: 1

...

  epoch    train_loss    valid_loss      dur
-------  ------------  ------------  -------
      1        4.2396        4.2318  77.2851
      2        4.1572        4.0760  78.9254
      3        4.0541        4.0428  79.0983
      4        4.0071        4.0507  78.8875
      5        3.9791        4.0120  79.5298
      6        3.9628        4.0018  79.2811
      7        3.9483        3.9941  79.8515
      8        3.9364        3.9899  79.5398
      9        3.9252        3.9884  79.9694
     10        3.9132        3.9889  79.7476
     11        3.9021        3.9795  80.4539
     12        3.8905        3.9694  80.4579
     13        3.8787        3.9550  80.7416
     14        3.8674        3.9468  80.5318
     15        3.8566        3.9402  80.9014
     16        3.8454        3.9303  81.0353
     17        3.8341        3.9296  80.7326
     18        3.8236        3.9207  80.2201
     19        3.8134        3.9174  80.7486
     20        3.8032        3.9174  81.6966
60 RMSE: 1

D:\Documents\anaconda3\envs\DataSciEnv\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'd', 'e', 'f', 'g', 'h', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


  epoch    train_loss    valid_loss      dur
-------  ------------  ------------  -------
      1        4.2385        4.2273  78.9924
      2        4.1531        4.0923  81.6567
      3        4.0585        4.0410  81.7755
      4        4.0240        4.0445  81.5857
      5        3.9996        4.0733  83.1641
      6        3.9807        4.0608  82.2281
      7        3.9653        4.0281  82.5797
      8        3.9526        4.0105  82.2950
      9        3.9415        3.9936  86.7859
     10        3.9311        3.9923  88.0426
     11        3.9206        3.9882  89.2056
     12        3.9096        3.9940  87.6016
     13        3.8988        3.9787  90.7241
     14        3.8877        3.9711  88.6361
     15        3.8764        3.9510  89.7344
     16        3.8655        3.9487  88.9202
     17        3.8552        3.9473  90.1149
     18        3.8440        3.9381  90.1402
     19        3.8322        3.9367  89.9144
     20        3.8209        3.9320  90.0512
80 RMSE: 1

D:\Documents\anaconda3\envs\DataSciEnv\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'd', 'e', 'f', 'g', 'h', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


  epoch    train_loss    valid_loss      dur
-------  ------------  ------------  -------
      1        4.2391        4.2305  91.1037
      2        4.1597        4.0905  91.6197
      3        4.0615        4.0427  93.8118
      4        4.0231        4.0615  87.9053
      5        3.9935        4.0699  86.8324
      6        3.9718        4.0330  87.2140
      7        3.9548        4.0091  86.8603
      8        3.9406        3.9947  87.4048
      9        3.9273        3.9923  89.1830
     10        3.9141        3.9760  89.6615
     11        3.9023        3.9703  91.9635
     12        3.8905        3.9540  91.1741
     13        3.8788        3.9364  98.2098
     14        3.8668        3.9312  100.5317
     15        3.8557        3.9287  102.9728
     16        3.8440        3.9212  102.4360
     17        3.8328        3.9139  103.2598
     18        3.8209        3.9078  104.4199
     19        3.8086        3.9056  103.0884
     20        3.7965        3.9020  101.5969
100

In [ ]:
optimum_params = {'net__lr': 0.0015539382932245303, 'net__max_epochs': 20, 'net__optimizer__momentum': 0.9375124227337543}
num_unitslist = [100,200,300,400,600,800,1000,2000]
rmselist = []
x_trainshape = 5842
for num in num_unitslist:
    class RegressorModule(nn.Module):
        def __init__(
                self,
                num_units=num,
                nonlin=F.relu,
        ):
            super(RegressorModule, self).__init__()
            self.num_units = num_units
            self.nonlin = nonlin

            self.dense0 = nn.Linear(x_trainshape, num_units)
            self.nonlin = nonlin
            self.dense1 = nn.Linear(num_units, num)
            self.output = nn.Linear(num, 1)

        def forward(self, X, **kwargs):
            X = self.nonlin(self.dense0(X))
            X = F.relu(self.dense1(X))
            X = self.output(X)
            return X

    pole_model = RegressorModule()

    net = NeuralNetRegressor(module=pole_model, max_epochs=optimum_params['net__max_epochs'], lr=optimum_params['net__lr'], optimizer__momentum=optimum_params['net__optimizer__momentum'], callbacks =[('earlystopping',EarlyStopping())])
    def typechange(x):
        return x.astype(dtype = np.float32)
    typetransform = FunctionTransformer(typechange)
    def inputneuron(x):
        x_trainshape = x.shape[1]
    #     return x_trainshape
    inputneuronnumber = FunctionTransformer(inputneuron)
    # pipe = Pipeline([('transform', DataFrameTransformer()),('net', net)])
    # pipe = Pipeline([('net', net)])
    pipe = Pipeline([("tfidf_vector_com", TfidfVectorizer(tokenizer=identity_tokenizer, input="array", lowercase=False, norm="l2", max_features=None, sublinear_tf=True, stop_words="english",token_pattern=r'[^\s]+')), ("typetransform", typetransform), ("net", net)])


    # net = skorch.NeuralNetClassifier(module=PoleNN, max_epochs=20, lr=0.1, criterion=torch.nn.NLLLoss)
    pipe.fit(X=x_train, y=y_train)
    y_pred = pipe.predict(x_test)
    rmse = mean_squared_error(y_test, y_pred, squared = False)
    print(num,'RMSE:', rmse)
    rmselist.append(rmse)

min_index = rmselist.index((min(rmselist)))
print("Optimum parameters:", num_unitslist[min_index],":",min(rmselist))

D:\Documents\anaconda3\envs\DataSciEnv\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'd', 'e', 'f', 'g', 'h', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


  epoch    train_loss    valid_loss      dur
-------  ------------  ------------  -------
      1        4.2400        4.2349  92.0159
      2        4.1679        4.0870  85.9619
      3        4.0655        4.0463  86.0232
      4        4.0245        4.0825  86.8104
      5        3.9895        4.0646  86.8074
      6        3.9685        4.0370  87.0012
      7        3.9520        4.0186  87.0961
      8        3.9378        4.0053  87.8753
      9        3.9241        3.9896  88.3648
     10        3.9113        3.9748  88.3633
     11        3.8975        3.9638  88.4677
     12        3.8857        3.9472  89.1810
     13        3.8727        3.9304  88.9892
     14        3.8604        3.9296  89.3078
     15        3.8487        3.9196  89.7979
     16        3.8358        3.9140  89.1660
     17        3.8247        3.9087  90.3907
     18        3.8131        3.9043  89.6515
     19        3.8003        3.9040  89.7783
     20        3.7892        3.9036  90.4496
100 RMSE: 

D:\Documents\anaconda3\envs\DataSciEnv\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'd', 'e', 'f', 'g', 'h', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


  epoch    train_loss    valid_loss      dur
-------  ------------  ------------  -------
      1        4.2404        4.2367  93.9621
      2        4.1729        4.0900  98.7672
      3        4.0641        4.0396  100.5494
      4        4.0201        4.0662  101.3396
      5        3.9864        4.0495  102.4774
      6        3.9644        4.0194  102.0998
      7        3.9469        4.0126  103.4984
      8        3.9320        3.9969  103.5693
      9        3.9183        3.9880  104.2886
     10        3.9043        3.9684  104.9279
     11        3.8921        3.9607  105.2096
     12        3.8799        3.9442  105.1257
     13        3.8668        3.9314  105.1587
     14        3.8548        3.9287  105.3505
     15        3.8428        3.9249  106.5552
     16        3.8290        3.9216  106.0408
     17        3.8154        3.9164  106.3554
     18        3.8012        3.9342  106.5722
     19        3.7894        3.9135  107.3315
     20        3.7737        3.9247  1

D:\Documents\anaconda3\envs\DataSciEnv\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'd', 'e', 'f', 'g', 'h', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


...

  epoch    train_loss    valid_loss       dur
-------  ------------  ------------  --------
      1        4.2399        4.2351  156.5172
      2        4.1675        4.0858  171.3881
      3        4.0518        4.0569  173.1822
      4        4.0007        4.0602  175.5458
      5        3.9710        4.0321  178.8005
      6        3.9506        4.0164  181.9323
      7        3.9329        3.9993  182.9203
      8        3.9172        3.9828  184.6835
      9        3.9022        3.9682  186.1550
     10        3.8869        3.9595  193.2193
     11        3.8714        3.9473  195.5246
     12        3.8554        3.9248  196.6167
     13        3.8388        3.9115  197.8036
     14        3.8197        3.9078  199.8828
     15        3.8015        3.9106  200.4695
     16        3.7852        3.8969  201.7175
     17        3.7648        3.9003  202.7969
     18        3.7457        3.9154  216.0335
     19        3.7268        3.9220  218.1577
     20        3.7103        3.942

D:\Documents\anaconda3\envs\DataSciEnv\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'd', 'e', 'f', 'g', 'h', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


  epoch    train_loss    valid_loss       dur
-------  ------------  ------------  --------
      1        4.2391        4.2313  213.8890
      2        4.1584        4.0764  229.8191
      3        4.0450        4.0637  233.7261
      4        3.9954        4.0567  229.0162
      5        3.9667        4.0350  232.6707
      6        3.9468        4.0185  234.1069
      7        3.9288        3.9933  240.0113
      8        3.9130        3.9696  243.5466
      9        3.8967        3.9589  256.2258
     10        3.8825        3.9462  240.1301
     11        3.8670        3.9271  258.7098
     12        3.8487        3.9254  236.7620
     13        3.8332        3.9296  236.2105
     14        3.8158        3.9235  233.9563
     15        3.7968        3.9062  236.0631
     16        3.7778        3.9125  238.1893
     17        3.7584        3.9163  244.1232
     18        3.7387        3.9474  244.0335
     19        3.7224        3.9516  247.8189
Stopping since valid_loss has not 

D:\Documents\anaconda3\envs\DataSciEnv\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'd', 'e', 'f', 'g', 'h', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


  epoch    train_loss    valid_loss       dur
-------  ------------  ------------  --------
      1        4.2381        4.2269  224.4839
      2        4.1489        4.0696  245.9677
      3        4.0397        4.0700  248.6130
      4        3.9918        4.0525  251.7387
      5        3.9643        4.0327  255.7215
      6        3.9435        4.0125  260.7315
      7        3.9271        3.9862  261.2781
      8        3.9108        3.9743  263.9237
      9        3.8942        3.9611  266.4313


In [21]:
profit = 0
for i, (real, pred) in enumerate(zip(y_test, y_pred)):
    if i == 0:
        pass
    else:
        realchange = y_test[i] - y_test[i-1]
        invest  = (y_pred[i] - y_pred[i-1])
        change = (invest * realchange)
        profit += change
print(profit)
profit = 0           
for i, (real, pred) in enumerate(zip(y_test, y_pred)):
    realmean = y_test.mean()
    predmean = y_pred.mean()
    realchange = real - realmean
    predchange = pred - predmean
    change = (predchange * realchange)
    profit += change
print(profit)
            
       

[71518.68]
[35764.336]


In [13]:
from scipy.stats import loguniform, uniform
print(uniform(20, 50))

In [8]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, cross_validate
from scipy.stats import loguniform, uniform
params = {
    'net__lr': [0.001,0.003,0.01,0.03,0.1,0.3,0.6,0.9,1],
    'net__max_epochs': [20,40,50],
    'net__optimizer__momentum': [0.95,0.96,0.97,0.98,0.99]
}

# specify parameters and distributions to sample from
param_dist = {'net__lr': loguniform(0.001,1),
              'net__max_epochs': [20,30,40,50],
              'net__optimizer__momentum': loguniform(0.9, 0.99)}

pipe = Pipeline([("typetransform", typetransform), ("net", net)])
tfidf = TfidfVectorizer(tokenizer=identity_tokenizer, input="array", lowercase=False, norm="l2", max_features=None, sublinear_tf=True, stop_words="english",token_pattern=r'[^\s]+')
x_train2 = tfidf.fit_transform(x_train)
x_trainshape = x_train2.shape[1]
class RegressorModule(nn.Module):
    def __init__(
            self,
            num_units=10,
            nonlin=F.relu,
    ):
        super(RegressorModule, self).__init__()
        self.num_units = num_units
        self.nonlin = nonlin

        self.dense0 = nn.Linear(x_trainshape, num_units)
        self.nonlin = nonlin
        self.dense1 = nn.Linear(num_units, 10)
        self.output = nn.Linear(10, 1)

    def forward(self, X, **kwargs):
        X = self.nonlin(self.dense0(X))
        X = F.relu(self.dense1(X))
        X = self.output(X)
        return X

pole_model = RegressorModule()
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(pole_model.parameters(), lr = 0.1)

net = NeuralNetRegressor(module=pole_model, max_epochs=30, lr=0.1, callbacks =[('earlystopping',EarlyStopping())])
# run randomized search
n_iter_search = 20
random_search = RandomizedSearchCV(pipe, param_distributions=param_dist,
                                   n_iter=n_iter_search)

start = time.time()
random_search.fit(x_train2, y_train)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time.time() - start), n_iter_search))
report(random_search.cv_results_)


# gs = GridSearchCV(pipe, params, n_jobs=-1)
# gs.fit(X=x_train, y=y_train);
# print(gs.best_score_, gs.best_params_)

  epoch    train_loss    valid_loss      dur
-------  ------------  ------------  -------
      1        3.8071        3.8260  51.2935
      2        3.7947        3.8399  51.5512
      3        3.7888        3.8498  51.6012
      4        3.7835        3.8568  52.3434
      5        3.7780        3.8655  51.8709
Stopping since valid_loss has not improved in the last 5 epochs.
  epoch    train_loss    valid_loss      dur
-------  ------------  ------------  -------
      1        3.8190        3.9146  51.4773
      2        3.8059        3.9112  51.8639
      3        3.7988        3.9343  52.0787
      4        3.7930        3.9281  52.5502
      5        3.7889        3.9176  52.0397
      6        3.7838        3.9211  51.9718
Stopping since valid_loss has not improved in the last 5 epochs.
  epoch    train_loss    valid_loss      dur
-------  ------------  ------------  -------
      1        3.8266        3.9289  51.4523
      2        3.8154        3.9252  52.3564
      3        

NameError: name 'report' is not defined

In [9]:
# https://scikit-learn.org/stable/auto_examples/model_selection/plot_randomized_search.html
# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})"
                  .format(results['mean_test_score'][candidate],
                          results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

report(random_search.cv_results_)
print(random_search.best_params_)

Model with rank: 1
Mean validation score: 0.099 (std: 0.011)
Parameters: {'net__lr': 0.0015539382932245303, 'net__max_epochs': 20, 'net__optimizer__momentum': 0.9375124227337543}

Model with rank: 2
Mean validation score: 0.098 (std: 0.011)
Parameters: {'net__lr': 0.0016770124951511836, 'net__max_epochs': 20, 'net__optimizer__momentum': 0.9457840905487397}

Model with rank: 3
Mean validation score: 0.097 (std: 0.011)
Parameters: {'net__lr': 0.002630441086119309, 'net__max_epochs': 40, 'net__optimizer__momentum': 0.9183456932254296}

{'net__lr': 0.0015539382932245303, 'net__max_epochs': 20, 'net__optimizer__momentum': 0.9375124227337543}


In [12]:
optimum_params = {'net__lr': 0.0015539382932245303, 'net__max_epochs': 20, 'net__optimizer__momentum': 0.9375124227337543}

net = NeuralNetRegressor(module=pole_model, max_epochs=optimum_params['net__max_epochs'], lr=optimum_params['net__lr'], optimizer__momentum=optimum_params['net__optimizer__momentum'], callbacks =[('earlystopping',EarlyStopping())])
pipe = Pipeline([("tfidf_vector_com", TfidfVectorizer(tokenizer=identity_tokenizer, input="array", lowercase=False, norm="l2", max_features=None, sublinear_tf=True, stop_words="english",token_pattern=r'[^\s]+')), ("typetransform", typetransform), ("net", net)])

pipe.fit(X=x_train, y=y_train)
y_pred = pipe.predict(x_test)

rmse = mean_squared_error(y_test, y_pred, squared = False)
print('Optimised RMSE:', rmse)

net = NeuralNetRegressor(module=pole_model, max_epochs=30, lr=0.1, callbacks =[('earlystopping',EarlyStopping())])

pipe.fit(X=x_train, y=y_train)
y_pred = pipe.predict(x_test)

rmse = mean_squared_error(y_test, y_pred, squared = False)
print('Original RMSE:', rmse)

D:\Documents\anaconda3\envs\DataSciEnv\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'd', 'e', 'f', 'g', 'h', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


  epoch    train_loss    valid_loss      dur
-------  ------------  ------------  -------
      1        3.8644        3.9511  68.8521
      2        3.8590        3.9415  68.8259
      3        3.8525        3.9416  68.9574
      4        3.8465        3.9420  69.1866
      5        3.8413        3.9412  69.2820
      6        3.8359        3.9374  69.7446
      7        3.8313        3.9369  68.9858
      8        3.8265        3.9419  69.9018
      9        3.8217        3.9387  71.4834
     10        3.8171        3.9361  73.6768
     11        3.8129        3.9373  74.3237
     12        3.8091        3.9397  66.4101
     13        3.8048        3.9407  65.8983
     14        3.8012        3.9412  66.0906
Stopping since valid_loss has not improved in the last 5 epochs.
Optimised RMSE: 1.9759188
Re-initializing optimizer because the following parameters were re-set: momentum.
  epoch    train_loss    valid_loss      dur
-------  ------------  ------------  -------
      1        3.

In [23]:
for act in ['Softmax','ReLU', 'Tanh', 'Sigmoid']:
    print(act)
    

    if act == 'Softmax':
        class RegressorModule(nn.Module):
            def __init__(
                    self,
                    num_units=10,
                    nonlin=F.relu,
            ):
                super(RegressorModule, self).__init__()
                self.num_units = num_units
                self.nonlin = nonlin

                self.dense0 = nn.Linear(x_trainshape, num_units)
                self.nonlin = nonlin
                self.dense1 = nn.Linear(num_units, 10)
                self.output = nn.Linear(10, 1)

            def forward(self, X, **kwargs):
                X = self.nonlin(self.dense0(X))
                X = F.Softmax(self.dense1(X))
                X = self.output(X)
                return X
        
    elif act == 'ReLU':        
        class RegressorModule(nn.Module):
            def __init__(
                    self,
                    num_units=10,
                    nonlin=F.relu,
            ):
                super(RegressorModule, self).__init__()
                self.num_units = num_units
                self.nonlin = nonlin

                self.dense0 = nn.Linear(x_trainshape, num_units)
                self.nonlin = nonlin
                self.dense1 = nn.Linear(num_units, 10)
                self.output = nn.Linear(10, 1)

            def forward(self, X, **kwargs):
                X = self.nonlin(self.dense0(X))
                X = F.relu(self.dense1(X))
                X = self.output(X)
                return X

    elif act == 'Tanh':         
        class RegressorModule(nn.Module):
            def __init__(
                    self,
                    num_units=10,
                    nonlin=F.relu,
            ):
                super(RegressorModule, self).__init__()
                self.num_units = num_units
                self.nonlin = nonlin

                self.dense0 = nn.Linear(x_trainshape, num_units)
                self.nonlin = nonlin
                self.dense1 = nn.Linear(num_units, 10)
                self.output = nn.Linear(10, 1)

            def forward(self, X, **kwargs):
                X = self.nonlin(self.dense0(X))
                X = F.Tanh(self.dense1(X))
                X = self.output(X)
                return X
            
    elif act == 'Sigmoid':  
        class RegressorModule(nn.Module):
            def __init__(
                    self,
                    num_units=10,
                    nonlin=F.relu,
            ):
                super(RegressorModule, self).__init__()
                self.num_units = num_units
                self.nonlin = nonlin

                self.dense0 = nn.Linear(x_trainshape, num_units)
                self.nonlin = nonlin
                self.dense1 = nn.Linear(num_units, 10)
                self.output = nn.Linear(10, 1)

            def forward(self, X, **kwargs):
                X = self.nonlin(self.dense0(X))
                X = F.sigmoid(self.dense1(X))
                X = self.output(X)
                return X
    net = NeuralNetRegressor(module=pole_model, max_epochs=optimum_params['net__max_epochs'], lr=optimum_params['net__lr'], optimizer__momentum=optimum_params['net__optimizer__momentum'], callbacks =[('earlystopping',EarlyStopping())])
    pipe = Pipeline([("tfidf_vector_com", TfidfVectorizer(tokenizer=identity_tokenizer, input="array", lowercase=False, norm="l2", max_features=None, sublinear_tf=True, stop_words="english",token_pattern=r'[^\s]+')), ("typetransform", typetransform), ("net", net)])

    pipe.fit(X=x_train, y=y_train)
    y_pred = pipe.predict(x_test)

    rmse = mean_squared_error(y_test, y_pred, squared = False)
    print(act,'RMSE:', rmse)


Softmax


D:\Documents\anaconda3\envs\DataSciEnv\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'd', 'e', 'f', 'g', 'h', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


  epoch    train_loss    valid_loss      dur
-------  ------------  ------------  -------
      1        3.8194        3.9282  69.1231
      2        3.8148        3.9283  68.3899
      3        3.8120        3.9277  69.3362
      4        3.8099        3.9283  68.5192
      5        3.8078        3.9283  69.8550
      6        3.8059        3.9294  69.3251
      7        3.8043        3.9297  69.3530
Stopping since valid_loss has not improved in the last 5 epochs.
Softmax RMSE: 1.9710398
ReLU


D:\Documents\anaconda3\envs\DataSciEnv\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'd', 'e', 'f', 'g', 'h', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


  epoch    train_loss    valid_loss      dur
-------  ------------  ------------  -------
      1        3.8013        3.9297  64.5240
      2        3.7999        3.9310  65.2932
      3        3.7986        3.9304  64.9056
      4        3.7972        3.9299  65.0265
      5        3.7960        3.9298  63.9895
Stopping since valid_loss has not improved in the last 5 epochs.
ReLU RMSE: 1.971134
Tanh


D:\Documents\anaconda3\envs\DataSciEnv\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'd', 'e', 'f', 'g', 'h', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


  epoch    train_loss    valid_loss      dur
-------  ------------  ------------  -------
      1        3.7939        3.9297  66.0624
      2        3.7926        3.9295  66.9165
      3        3.7914        3.9297  66.3971
      4        3.7902        3.9299  66.9555
      5        3.7890        3.9308  67.7387
Stopping since valid_loss has not improved in the last 5 epochs.
Tanh RMSE: 1.9713187
Sigmoid


D:\Documents\anaconda3\envs\DataSciEnv\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'd', 'e', 'f', 'g', 'h', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


  epoch    train_loss    valid_loss      dur
-------  ------------  ------------  -------
      1        3.7869        3.9310  64.5859
      2        3.7855        3.9313  65.1004
      3        3.7848        3.9307  65.5130
      4        3.7837        3.9308  65.4021
      5        3.7828        3.9299  64.1793
      6        3.7819        3.9312  64.6079
      7        3.7805        3.9313  65.8117
      8        3.7799        3.9309  64.4700
      9        3.7786        3.9310  64.5439
Stopping since valid_loss has not improved in the last 5 epochs.
Sigmoid RMSE: 1.9714288


In [ ]:
dftest = pd.DataFrame(y_test)
dftest['index'] = list(range(103427))
dfpred = pd.DataFrame(y_pred)
dfpred['index'] = list(range(103427))
fix, ax = plt.subplots()
sns.kdeplot(x=dftest['index'],y=dftest[0], cumulative=True, color='orange', label='real')
sns.kdeplot(x=dfpred['index'],y=dfpred[0], cumulative=True, color='b', label='pred')

plt.show()

In [ ]:

from sklearn.ensemble import RandomForestRegressor
text_classifier = RandomForestRegressor(n_estimators=100, random_state=0)  
text_classifier.fit(x_train, y_train)


predictions = text_classifier.predict(x_test)
 
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
 
print(confusion_matrix(y_test,predictions))  
print(classification_report(y_test,predictions))  
print(accuracy_score(y_test, predictions))

In [73]:
cross_val_score.get_params()

AttributeError: 'function' object has no attribute 'get_params'

In [8]:
y = merged['Change']

X = processed_tweets

data = pd.DataFrame()
data['text'] = processed_tweets
data['y'] = y.tolist()


from sklearn.model_selection import train_test_split  
data1 = data.sample(frac=0.2)
X1 = data1['text'].tolist()
y2 = np.array(data1['y'])
x_trainsvm, x_testsvm, y_trainsvm, y_testsvm = train_test_split(X1, y2, test_size=0.2, random_state=0)


In [9]:
from sklearn.svm import LinearSVR, SVR

def identity_tokenizer(text):
    return text


# y_trainsvm = np.array(y_train).ravel()
# y_testsvm = np.array(y_test).ravel()

#Create the SVM model

start = time.time()
# regressor = LinearSVR(random_state = 0)
regressor = SVR()
#Fit the model for the data
pipe = Pipeline([("tfidf_vector_com", TfidfVectorizer(tokenizer=identity_tokenizer, input="array", lowercase=False, norm="l2", max_features=None, sublinear_tf=True, stop_words="english",token_pattern=r'[^\s]+')), ("regressor", regressor)])

pipe.fit(x_trainsvm, y_trainsvm)

#Make the prediction
y_predsvm = pipe.predict(x_testsvm)

end = time.time()
print("Elapsed Time: ", (end-start),"\n")

D:\Documents\anaconda3\envs\DataSciEnv\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'd', 'e', 'f', 'g', 'h', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


Elapsed Time:  1705.32363986969 



In [11]:
rmse = mean_squared_error(y_testsvm, y_predsvm, squared = False)
print("RMSE:",rmse)
r2 = r2_score(y_testsvm,y_predsvm)
print("R-Squared:",r2)

RMSE: 1.977553460848178
R-Squared: 0.0333595799537606


In [ ]:
accuracies = cross_val_score(estimator = regressor, X = x_trainsvm, y = y_trainsvm, cv = 10)
end = time.time()
# print("Fraction: ", x)
# print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
# print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))
mse = mean_squared_error(y_test,y_predsvm)
print("Mean Squared Error: {}".format(mse))
r2 = r2_score(y_test,y_predsvm)
print("R-squared: {}".format(r2))
print("")
print("Elapsed Time: ", (end-start),"\n")

D:\Documents\anaconda3\envs\DataSciEnv\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'd', 'e', 'f', 'g', 'h', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


In [31]:
mse = mean_squared_error(y_test,y_predsvm)
print("Mean Squared Error: {}".format(mse))
r2 = r2_score(y_test,y_predsvm)
print("R-squared: {}".format(r2))

4.00098072231255
Mean Squared Error: 4.00098072231255
R-squared: 0.025518011281076225
